# Scenario 4. Robustness (regression design)

## Strategy 2. Mis-specification (Table 12)

In [1]:
R.version
library(tidyverse)
library(caret)
library(randomForest)
library(reticulate)
np <- import("numpy")

               _                           
platform       x86_64-pc-linux-gnu         
arch           x86_64                      
os             linux-gnu                   
system         x86_64, linux-gnu           
status                                     
major          3                           
minor          5.3                         
year           2019                        
month          03                          
day            11                          
svn rev        76217                       
language       R                           
version.string R version 3.5.3 (2019-03-11)
nickname       Great Truth                 

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:dplyr’:

    combine


The following object is masked from ‘package:ggplot2’:

    margin




### Configuration

In [2]:
data_path = "./data/simulation/s5"
path_genus = "./data/genus48"

y_path = sprintf('%s/%s', data_path, 'y.csv')
tree_info_path = './data/genus48/genus48_dic.csv'
count_path = './data/simulation/count'
count_list_path = './data/simulation/gcount_list.csv'
idx_path = './data/simulation/s1/idx.csv'

num_classes = 0 # regression
tree_level_list = c('Genus', 'Family', 'Order', 'Class', 'Phylum')

In [3]:
# # Read phylogenetic tree information

# phylogenetic_tree_info = read.csv(tree_info_path)
# phylogenetic_tree_info = phylogenetic_tree_info %>% select(tree_level_list)

# print(sprintf('Phylogenetic tree level list: %s', str_c(phylogenetic_tree_info %>% colnames, collapse = ', ')))

### Read dataset

#### Read training, test dataset

In [4]:
read_dataset <- function(x_path, y_path, sim){
    print(str_c('Load data for repetition ', sim))
    x = read.csv(x_path)
    y = read.csv(y_path)[,sim]
    x = (x - max(x)) / (max(x) - min(x))

    idxs = idxs_total[, sim]
    remain_idxs = setdiff(seq(1, dim(x)[1]), idxs)

    x_train = x[idxs,]
    x_test = x[remain_idxs,]
    y_train = y[idxs]
    y_test = y[remain_idxs]
    
    return (list(x_train, x_test, y_train, y_test))
}

In [5]:
idxs_total = read.csv(idx_path)
number_of_fold = dim(idxs_total)[2]; number_of_fold
x_list = read.csv(count_list_path, header = FALSE)
x_path = x_list$V1 %>% sprintf('%s/%s', count_path, .)

[1] 1000

#### Read true tree weight

In [6]:
tw_1 = np$load(sprintf('%s/tw_1.npy', data_path))

## Random Forest

### Importance type

* See <https://stats.stackexchange.com/questions/92419/relative-importance-of-a-set-of-predictors-in-a-random-forests-classification-in>

Here are the definitions of the variable importance measures.

- `type=1`: **Mean decrease in accuracy**
    - The first measure is computed from permuting Out-of-bag (OOB) data.
    - For each tree, the prediction error on the out-of-bag portion of the data is recorded (error rate for classification, MSE for regression). Then the same is done after permuting each predictor variable. 
    - The difference between the two are then averaged over all trees, and normalized by the standard deviation of the differences.
    - If the standard deviation of the differences is equal to 0 for a variable, the division is not done (but the average is almost always equal to 0 in that case).

- `type=2`: **Mean decrease in node impurity**
    - The second measure is the total decrease in node impurities from splitting on the variable, averaged over all trees. 
    - For classification, the node impurity is measured by the Gini index. 
    - For regression, it is measured by residual sum of squares.
    
### Feature selection

* `vi_f`: variable importance by Gini importance (`type=2`)
* `relative_vi_f` : relative variable importance
* `thrd`: threshold for relative variable importance
* Select features which have relative variable importance `relative_vi_f` equal or larger than threshold `thrd`

### Simulate for all $n$

In [7]:
random_forest_res <- function(fold, importance_type=2, fs_thrd = 0.1){
    print(sprintf('-----------------------------------------------------------------'))
    print(sprintf('Random Forest computation for %dth repetition', fold))

    dataset = read_dataset(x_path[fold], y_path, fold)
    x_train = dataset[[1]]
    x_test = dataset[[2]]
    y_train = dataset[[3]]
    y_test = dataset[[4]]

    fit.rf <- randomForest(y_train~.,data=x_train, ntree=1000,  mtry=10, importance=TRUE)
    train.pred <- fit.rf$predicted
    test.pred <- predict(fit.rf,x_test)

    train.mse <- mean((y_train - train.pred)^2)
    train.cor <- cor(y_train, train.pred)

    test.mse <- mean((y_test - test.pred)^2)
    test.cor <- cor(y_test, test.pred)
    
    # Feature selection
    ## variable importance
    vi_f = importance(fit.rf, type=importance_type)
    relative_vi_f <- vi_f / sum(vi_f)
    selected_genus <- ifelse(relative_vi_f >= fs_thrd, 1, 0)
    
    order <- order(relative_vi_f, decreasing = TRUE)
    sorted_relative_vi_f <- relative_vi_f[order]
    names(sorted_relative_vi_f) <- colnames(x_train)[order]
    print(sorted_relative_vi_f)

    fold_genus = apply(tw_1[fold,,], 1, sum)
    names(fold_genus) <- x_train %>% colnames

    fs_conf_table <- table(selected_genus, fold_genus)
    
    fs_sensitivity <- sensitivity(fs_conf_table) 
    fs_specificity <- specificity(fs_conf_table)
    fs_gmeasure <- sqrt(fs_sensitivity*fs_specificity)
    fs_accuracy <- sum(diag(fs_conf_table))/sum(fs_conf_table)

    print(sprintf('Train mse: %s, Train Correlation: %s', train.mse, train.cor))
    print(sprintf('Test mse: %s, Test Correlation: %s', test.mse, test.cor))
    print(sprintf('FS sensitivity: %s, FS sensitivity: %s, FS gmeasure: %s, FS accuracy: %s',
                  fs_sensitivity, fs_specificity, fs_gmeasure, fs_accuracy))
    
    return (c(train.mse, train.cor, test.mse, test.cor, fs_sensitivity, fs_specificity, fs_gmeasure, fs_accuracy))
}

In [8]:
set.seed(100)
# res <- sapply(seq(1,1), random_forest_res)
res <- sapply(seq(1,10), random_forest_res)
# res <- sapply(seq(1,number_of_fold), random_forest_res)

[1] "-----------------------------------------------------------------"
[1] "Random Forest computation for 1th repetition"
[1] "Load data for repetition 1"
                   Tropheryma                        Rothia 
                 0.5073680715                  0.0890775066 
                  Actinomyces                  Oribacterium 
                 0.0660210201                  0.0201370656 
                Streptococcus                Pseudonocardia 
                 0.0162452076                  0.0147550297 
            Propionibacterium     TM7_genera_incertae_sedis 
                 0.0139086113                  0.0132158747 
                Fusobacterium                       Gemella 
                 0.0127655529                  0.0127608080 
                    Catonella               Corynebacterium 
                 0.0123463052                  0.0121985501 
                    Treponema                 Porphyromonas 
                 0.0115507124                  0.01

In [9]:
results_table = res %>% t %>% data.frame
colnames(results_table) = c('Training MSE', 'Training Correlation', 'Test MSE', 'Test Correlation', 
                            'Taxa selection sensitivity','Taxa selection sensitivity',
                            'Taxa selection gmeasure', 'Taxa selection accuracy')
results_table

Training MSE,Training Correlation,Test MSE,Test Correlation,Taxa selection sensitivity,Taxa selection sensitivity,Taxa selection gmeasure,Taxa selection accuracy
0.3712227,0.9208287,0.4710033,0.9068924,1,0.05263158,0.2294157,0.625
0.3532114,0.9094144,0.2712462,0.9521910,1,0.05263158,0.2294157,0.625
0.3444388,0.9188057,0.3708561,0.9107067,1,0.05263158,0.2294157,0.625
0.3549541,0.9169658,0.2895237,0.8888439,1,0.05263158,0.2294157,0.625
0.3612656,0.9161159,0.3469143,0.9252518,1,0.05263158,0.2294157,0.625
0.2914072,0.9342323,0.3684919,0.8896367,1,0.05263158,0.2294157,0.625
0.3263724,0.9182612,0.3320707,0.9021671,1,0.05263158,0.2294157,0.625
0.3361833,0.9169895,0.4809564,0.8935988,1,0.05263158,0.2294157,0.625
0.3948494,0.9178278,0.3656586,0.9124647,1,0.05263158,0.2294157,0.625
0.3575917,0.9106707,0.4081950,0.9000736,1,0.05263158,0.2294157,0.625


In [10]:
print('Mean')
apply(results_table, 2, mean)

[1] "Mean"


Training MSE       Training Correlation 
                0.34914965                 0.91801120 
                  Test MSE           Test Correlation 
                0.37049163                 0.90818268 
Taxa selection sensitivity Taxa selection sensitivity 
                1.00000000                 0.05263158 
   Taxa selection gmeasure    Taxa selection accuracy 
                0.22941573                 0.62500000

In [11]:
print('SD')
apply(results_table, 2, sd)

[1] "SD"


Training MSE       Training Correlation 
               0.027663159                0.006710764 
                  Test MSE           Test Correlation 
               0.068468671                0.019110888 
Taxa selection sensitivity Taxa selection sensitivity 
               0.000000000                0.000000000 
   Taxa selection gmeasure    Taxa selection accuracy 
               0.000000000                0.000000000